In [ ]:
import os
from PIL import Image, ImageFilter, ImageEnhance
import pandas as pd

In [ ]:
df = pd.read_csv("blue_cone.csv")

In [ ]:
df.describe().loc["count"]

##### 1. Current instances present in old set are: 17485
##### 2. CSV data instances are: 86116
##### 3. (86116 - 2568) = 83548
##### 4. Required from csv are : 86116 - 83548 = 2568 rows
##### 5. 2568 + 17485 = 20053

In [ ]:
# df_b = df.iloc[-1264:].reset_index(drop=True)
df_b = df.iloc[0:2568].reset_index(drop=True)

In [ ]:
df_b.describe().loc["count"]

In [ ]:
df_b.index

In [ ]:
df_b.head(3)

### load image and crop image

In [ ]:
print("image path", df_b.image_path[0])
print("label path", df_b.path[0])

In [ ]:
def crop_and_save(image_path, label_path, box, img_output_folder, label_output_folder, class_label, counter):
    img = Image.open(image_path)
    # cropped_img = img.crop(box)
    # resized_img = cropped_img.resize((640, 640), resample=Image.BICUBIC)
    # enhancer = ImageEnhance.Contrast(resized_img)
    # contrasted_img = enhancer.enhance(1.5)
    # sharpened_img = contrasted_img.filter(ImageFilter.SHARPEN)

    # Get the new bounding box coordinates in the cropped image
    # new_box = (0, 0, box[2] - box[0], box[3] - box[1])
    # new_box = (0, 0, 640, 640)
    new_box, cropped_img = convert_to_normalized_coordinates(box, img)
    
    # Create output folder if not exists
    os.makedirs(img_output_folder, exist_ok=True)
    os.makedirs(label_output_folder, exist_ok=True)
    
    # Save cropped image and labels to output folder
    img_output_folder = os.path.join(img_output_folder, os.path.basename(image_path))
    label_output_folder = os.path.join(label_output_folder, f"{os.path.basename(image_path)}.txt")

    # Image new name.
    image_new_name = img_new_name(img_output_folder, counter)
    
    cropped_img.save(image_new_name)

    # Write new bounding box coordinates to a text file
    text_file_path = label_output_folder
    label_name_update = label_new_name(text_file_path, counter)
    with open(text_file_path, 'w') as text_file:
        text_file.write(f"{class_label} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}")
    os.rename(text_file_path, label_name_update)

    print("Saved image: ", image_new_name)
    print("Saved label: ", label_name_update)

def img_new_name(path, counter):
    ext = os.path.basename(path).split(".")[-1]
    dir_name = os.path.dirname(path)
    parent_dir_name = os.path.basename(os.path.dirname(os.path.dirname(path)))
    new_image_path = os.path.join(dir_name, str(parent_dir_name) +"_"+ str(counter) +"_"+ str(ext) +"."+ str(ext))
    return new_image_path

def label_new_name(path, counter):
    ext_1 = os.path.basename(path).split(".")[-1]
    ext_2 = os.path.basename(path).split(".")[-2]
    parent_dir_name = os.path.basename(os.path.dirname(os.path.dirname(path)))
    dir_name = os.path.dirname(path)
    new_label_path = os.path.join(dir_name, str(parent_dir_name) +"_"+ str(counter) +"_"+ str(ext_2) +"."+ str(ext_1))
    return new_label_path


# def clamp(value, min_value, max_value):
#     return max(min(value, max_value), min_value)

# def new_box_coordinates(box, img):
#     # Extracting values from the box
#     left, upper, right, lower = box

#     # Original bounding box in pixel coordinates
#     original_box = (
#         clamp(left, 0, img.width),
#         clamp(upper, 0, img.height),
#         clamp(right, 0, img.width),
#         clamp(lower, 0, img.height)
#     )
#     print("Original Bounding Box in Pixel Coordinates:", original_box)

#     # Apply cropping
#     cropped_img = img.crop(original_box)

#     new_box = (
#         original_box[0] / img.width,
#         original_box[1] / img.height,
#         (original_box[2] - original_box[0]) / img.width,
#         (original_box[3] - original_box[1]) / img.height
#     )

#     print("New Bounding Box in Normalized Coordinates:", new_box)
#     return new_box, cropped_img

def convert_to_normalized_coordinates(box, img):
    cropped_img = img.crop(box)
    image_width, image_height = 1000, 1500
    box_x1, box_y1, box_x2, box_y2 = box
    
    # Calculate normalized coordinates
    x_center = (box_x1 + box_x2) / (2.0 * image_width)
    y_center = (box_y1 + box_y2) / (2.0 * image_height)
    width = (box_x2 - box_x1) / image_width
    height = (box_y2 - box_y1) / image_height

    # Format the result as a string
    new_box = f"{x_center:.3f} {y_center:.3f} {width:.3f} {height:.3f}"
    print("New Bounding Box in Normalized Coordinates:", new_box)
    return new_box, cropped_img


# def clamp(value, min_value, max_value):
#     return max(min(value, max_value), min_value)

# def new_box_coordinates(box, img):
#     # Extracting values from the box
#     left, upper, right, lower = box

#     # Normalize pixel coordinates
#     original_width, original_height = img.size  # Replace with the actual dimensions of the original image

#     x_center = (left + right) / (2 * original_width)
#     y_center = (upper + lower) / (2 * original_height)
#     width = (right - left) / original_width
#     height = (lower - upper) / original_height

#     # Original bounding box in normalized form
#     original_box = (
#         clamp(x_center - width / 2, 0, 1),
#         clamp(y_center - height / 2, 0, 1),
#         clamp(x_center + width / 2, 0, 1),
#         clamp(y_center + height / 2, 0, 1)
#     )
#     print("Original Bounding Box in Normalized Coordinates:", original_box)

#     # Convert normalized coordinates to pixel coordinates after cropping
#     left = int(original_box[0] * original_width)
#     upper = int(original_box[1] * original_height)
#     right = int(original_box[2] * original_width)
#     lower = int(original_box[3] * original_height)

#     # Clamp coordinates to ensure they are within the image boundaries
#     left = clamp(left, 0, original_width - 1)
#     upper = clamp(upper, 0, original_height - 1)
#     right = clamp(right, 0, original_width - 1)
#     lower = clamp(lower, 0, original_height - 1)

#     # Check if the cropped region is valid
#     if left >= right or upper >= lower:
#         raise ValueError(f"Invalid bounding box or dimensions for cropping. Bounding box: {original_box}, Image size: {original_width}x{original_height}")

#     # Apply cropping
#     cropped_img = img.crop((left, upper, right, lower))

#     new_box = (0, 0, right - left, lower - upper)

#     print("New Bounding Box in Pixel Coordinates:", new_box)
#     return new_box, cropped_img



In [ ]:
# Specify output folder for cropped images and labels
img_output_folder = 'blue_crop/images'
label_output_folder = 'blue_crop/labels'

# Iterate through rows and crop/save images
for index, row in df_b.iterrows():
    image_path = row['image_path']
    label_path = row['path']
    class_label = row["class_label"]
    box = (row['box_x1'], row['box_y1'], row['box_x2'], row['box_y2'])
    
    crop_and_save(image_path, label_path, box, img_output_folder, label_output_folder, class_label, index)
    
    print("Cropping and saving complete.", index)

#### Verify Bonding box on sample image

In [ ]:
from PIL import Image, ImageDraw
from IPython.display import display

# Open the image
img = Image.open('blue_crop/images/blue_crop_16_png.png')

draw = ImageDraw.Draw(img)
normalized_coordinates="0.761,0.713,0.081,0.075"
# Split normalized coordinates
x_center, y_center, width, height = map(float, normalized_coordinates.split(','))

# Calculate box coordinates in pixel values
box_x1 = int((x_center - width / 2) * img.width)
box_y1 = int((y_center - height / 2) * img.height)
box_x2 = int((x_center + width / 2) * img.width)
box_y2 = int((y_center + height / 2) * img.height)

# Draw rectangle on the image
draw.rectangle([box_x1, box_y1, box_x2, box_y2], outline=(0, 255, 0), width=2)

# Display the image in the notebook
display(img)
